In [ ]:
!sudo apt update
!sudo apt install -y pciutils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [62.5 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,234 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu 

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import threading
import subprocess
import time

def run_ollama_serve():
    subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)  # Allows service to initialize


In [ ]:
!ollama pull llama3.2


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏ 3.8 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   2% ▕▏  49 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   3% ▕▏  63 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   5% ▕▏ 109 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   7% ▕▏ 139 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   7% ▕▏ 149 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   8% ▕▏ 163 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   9% ▕▏ 191 MB/

In [ ]:
!pip install langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.7 MB/s eta 0:00:00


In [ ]:
import os
os.environ['OLLAMA_HOST'] = '127.0.0.1:11434'


In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama3.2")
response = llm.invoke("tell me a joke")
print(response)


<ipython-input-7-9a3c0c478701>:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


Here's one:

What do you call a fake noodle?

An impasta.


In [ ]:
!pip install fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.3 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok, conf
from langchain_community.llms import Ollama
import os

# Initialize the Flask app
app = Flask(__name__)

# Configure the Ollama environment
os.environ['OLLAMA_HOST'] = '127.0.0.1:11434'

# Initialize the Llama3.2 model
llm = Ollama(model="llama3.2")

# Define the endpoint for the model
@app.route('/invoke', methods=['POST'])
def invoke():
    try:
        data = request.json
        prompt = data.get('prompt', '')
        if not prompt:
            return jsonify({"error": "Prompt is required"}), 400

        # Generate a response from the model
        response = llm.invoke(prompt)
        return jsonify({"response": response})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Authenticate and start the Ngrok tunnel
def start_ngrok(auth_token):
    conf.get_default().auth_token = auth_token
    public_url = ngrok.connect(5000)
    print(f"Public URL: {public_url}")
    return public_url

if __name__ == '__main__':
    # Replace 'your-ngrok-auth-token' with your actual Ngrok authentication token
    NGROK_AUTH_TOKEN = "enter_ngrok_auth_token"

    if not NGROK_AUTH_TOKEN:
        print("Error: Ngrok authentication token is required")
        exit(1)

    public_url = start_ngrok(NGROK_AUTH_TOKEN)

    print(f"The public API endpoint is available at: {public_url}/invoke")

    # Start the Flask app
    app.run(port=5000)

Public URL: NgrokTunnel: "https://d5bd-34-145-95-7.ngrok-free.app" -> "http://localhost:5000"
The public API endpoint is available at: NgrokTunnel: "https://d5bd-34-145-95-7.ngrok-free.app" -> "http://localhost:5000"/invoke
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/Jan/2025 22:40:08] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Jan/2025 22:40:08] "GET /apple-touch-icon-precomposed.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Jan/2025 22:40:08] "GET /apple-touch-icon.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Jan/2025 22:41:52] "POST /invoke HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jan/2025 22:42:41] "POST /invoke HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jan/2025 22:44:50] "POST /invoke HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jan/2025 22:47:35] "POST /invoke HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jan/2025 22:48:38] "POST /invoke HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Jan/2025 22:59:07] "POST /invoke HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 -